## Translation Inference using Online Endpoints

This sample shows how to deploy `translation` type models to an online endpoint for inference.

### Task
`translation` converts a sequence of text from one language to another. It is one of several tasks you can formulate as a sequence-to-sequence problem, a powerful framework for returning some output from an input, like translation or summarization. `translation` systems are commonly used for translation between different language texts, but it can also be used for speech or some combination in between like text-to-speech or speech-to-text.

### Model
Models that can perform the `translation` task are tagged with `task: translation`. We will use the `t5-small` model in this notebook. If you opened this notebook from a specific model card, remember to replace the specific model name. If you don't find a model that suits your scenario or domain, you can discover and [import models from HuggingFace hub](../../import/import-model-from-huggingface.ipynb) and then use them for inference. 

### Inference data
We will use the [wmt16 (ro-en)](https://huggingface.co/datasets/wmt16) dataset. A copy of this dataset is available in the [wmt16-en-ro-dataset](./wmt16-en-ro-dataset/) folder. 

### Outline
* Set up pre-requisites.
* Pick a model to deploy.
* Prepare data for inference. 
* Deploy the model for real time inference.
* Test the endpoint
* Clean up resources.

### 1. Set up pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry

In [1]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
    ClientSecretCredential,
)
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = MLClient(
    credential,
    subscription_id="ea4faa5b-5e44-4236-91f6-5483d5b17d14",
    resource_group_name="amyharrispersonal",
    workspace_name="amyharris-canary",
)
# the models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml-preview"
registry_ml_client = MLClient(credential, registry_name="azureml-preview")

Class FeatureStoreOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureSetOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureStoreEntityOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### 2. Pick a model to deploy

Browse models in the Model Catalog in the AzureML Studio, filtering by the `translation` task. In this example, we use the `t5-small` model. If you have opened this notebook for a different model, replace the model name and version accordingly. 

In [2]:
model_name = "t5-small"
model_version = "4"
foundation_model = registry_ml_client.models.get(model_name, model_version)
print(
    "\n\nUsing model name: {0}, version: {1}, id: {2} for inferencing".format(
        foundation_model.name, foundation_model.version, foundation_model.id
    )
)



Using model name: t5-small, version: 4, id: azureml://registries/azureml-preview/models/t5-small/versions/4 for inferencing


### 3. Prepare data for inference.

A subset of the wmt16-en-ro dataset is available in the [wmt16-en-ro-dataset](./wmt16-en-ro-dataset/) folder.  The next few cells show basic data preparation:
* Visualize some data rows
* Save few samples in the format that can be passed as input to the online-inference endpoint.

In [3]:
# load the ./wmt16-en-ro-dataset/train_100.jsonl file into a pandas dataframe and show the first 5 rows
import pandas as pd

pd.set_option(
    "display.max_colwidth", 0
)  # set the max column width to 0 to display the full text
train_df = pd.read_json("./wmt16-en-ro-dataset/train_100.jsonl", lines=True)
train_df.head()

,en,ro
0,"That means that diversification of pipelines, sources and, of course, of different energy - as has been said here - will be the way to go in the future.","Aceasta înseamnă că soluţia care trebuie urmată în viitor este cea a diversificării conductelor, a surselor şi, desigur, a tipurilor de energie - astfel cum s-a menţionat în cadrul acestei discuţii."
1,Approval of the minutes of the previous sitting: see Minutes,Aprobarea procesului-verbal al şedinţei anterioare: consultaţi procesul-verbal
2,E-commerce is an excellent commercial opportunity for small and medium-sized enterprises and for young entrepreneurs.,Comerţul electronic reprezintă o excelentă oportunitate comercială pentru IMM-uri şi tinerii întreprinzători.
3,Approval of Minutes of previous sitting: see Minutes,Aprobarea procesului-verbal al şedinţei precedente: a se vedea procesul-verbal
4,Composition of committees and delegations: see Minutes,Componenţa comisiilor şi a delegaţiilor: a se vedea procesul-verbal


### 4. Deploy the model to an online endpoint
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model.

In [4]:
import time, sys
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    OnlineRequestSettings,
)

# Create online endpoint - endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
timestamp = int(time.time())
online_endpoint_name = "translation-" + str(timestamp)
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for "
    + foundation_model.name
    + ", for translation task",
    auth_mode="key",
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [5]:
# create a deployment
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=foundation_model.id,
    instance_type="Standard_DS2_v2",
    instance_count=1,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=60000,
    ),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {"demo": 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

Instance type Standard_DS2_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint translation-1684194083 exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


.............................................................................................................................

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://translation-1684194083.eastus2euap.inference.ml.azure.com/score', 'openapi_uri': 'https://translation-1684194083.eastus2euap.inference.ml.azure.com/swagger.json', 'name': 'translation-1684194083', 'description': 'Online endpoint for t5-small, for translation task', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/ea4faa5b-5e44-4236-91f6-5483d5b17d14/resourcegroups/amyharrispersonal/providers/microsoft.machinelearningservices/workspaces/amyharris-canary/onlineendpoints/translation-1684194083', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ea4faa5b-5e44-4236-91f6-5483d5b17d14/providers/Microsoft.MachineLearningServices/locations/eastus2euap/mfeOperationsStatus/oe:c76e6446-545b-4141-80f9-e8ad59c471f2:aa28d203-b416-40e3-9461-0dce81cc07ce?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ea4faa5b-5e44

### 5. Test the endpoint with sample data

We will fetch some sample data from the test dataset and submit to online endpoint for inference. We will then show the display the scored labels alongside the ground truth labels

In [6]:
import json
import os

# read the ./wmt16-en-ro-dataset/train_100.jsonl file into a pandas dataframe
df = pd.read_json("./wmt16-en-ro-dataset/train_100.jsonl", lines=True)
# escape single and double quotes in the text column
df["en"] = df["en"].str.replace("'", "\\'").str.replace('"', '\\"')
# pick 1 random row
sample_df = df.sample(1)
# create a json object with the key as "inputs" and value as a list of values from the en column of the sample_df dataframe
test_json = {
    "inputs": {"input_string": sample_df["en"].tolist()},
    "parameters": {"task_type": "translation_en_to_ro"},
}
# save the json object to a file named sample_score.json in the ./wmt16-en-ro-dataset folder
with open(os.path.join(".", "wmt16-en-ro-dataset", "sample_score.json"), "w") as f:
    json.dump(test_json, f)
sample_df.head()

,en,ro
23,It is also an industrial chemical that is actually used a lot in the pharmaceutical industry.,"Acesta este, de asemenea, o substanţă chimică industrială care este utilizată de fapt foarte mult în industria farmaceutică."


In [7]:
# score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="demo",
    request_file="./wmt16-en-ro-dataset/sample_score.json",
)
print("raw response: \n", response, "\n")
# convert the json response to a pandas dataframe
response_df = pd.read_json(response)
response_df.head()

raw response: 
 [{"0": "Este, de asemenea, o substan\u021b\u0103 chimic\u0103 industrial\u0103 care este utilizat\u0103 foarte mult \u00een industria farmaceutic\u0103."}] 



,0
0,"Este, de asemenea, o substanță chimică industrială care este utilizată foarte mult în industria farmaceutică."


In [8]:
# compare the predicted translation with the ground truth translation
response_df.rename(columns={"translation_text": "predicted_translation"}, inplace=True)
response_df["ground_truth_translation"] = sample_df["ro"].tolist()
response_df.head()

,0,ground_truth_translation
0,"Este, de asemenea, o substanță chimică industrială care este utilizată foarte mult în industria farmaceutică.","Acesta este, de asemenea, o substanţă chimică industrială care este utilizată de fapt foarte mult în industria farmaceutică."


### 6. Delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint

In [9]:
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()

........................................................................................................